In [111]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [112]:
# given dataset with lat,lon convert to grid position
def fit_point_to_grid(point,square_pad):
    """calculate the grid square that a coordinate would belong to. 
    
    the grid represents positions between 180,-180 and -180,180
        Each square being square_pad x square_pad across.

    """
    
    # check if the square_pad is not valid (will not divide the grid into equal intervals)
    if 360 / square_pad % 1 != 0:
        raise ValueError(f"""{square_pad} is not a valid value for square_pad. 
                         (360 must be divisible by square_pad to ensure equal grid square sizes)"""
                        )      
    
    lat,lon = point
    lat_idx = int((lat+180) / square_pad)
    lon_idx = int((lon+180) / square_pad)
    
    return lat_idx, lon_idx


def fit_all_points_to_grid(df,square_pad, weight=1):
    df_grid = {}
    if not all(name in df.columns for name in ["lat","lon"]):
        return None
    
    for point in df[["lat","lon"]].values:
        grid_point = fit_point_to_grid(point, square_pad=square_pad)
        
        if grid_point in df_grid:
            df_grid[grid_point] += weight
        else:
            df_grid[grid_point] = weight
    
    return df_grid
# TODO make this projected coordinates

In [113]:
# generate surrounding weights of each point
def add_radius_to_grid_dict(grid_dict, square_pad, radius=5):
    """ add a radius around each point in the grid_dict dict that decreases in weight
    as it moves further away from the point. 
    
    currently radius is just the surrounding indexes of a point. other solution using actual
    circle radius would require converting to polar coordinates, and rastering the vector 
    circle back into a grid representaton. 
    
    """
    
    # do nothing if radius is 0 or less
    if radius <= 0:
        return
    
    # calculate the boundaries of the grid 
    # (grid is not an object), so it's shape must be dynamically calculated
    x_max, y_max = fit_point_to_grid((180,180), square_pad=square_pad)
    x_min, y_min = (0,0)
    
    # store values in a temporary dict first, to keep the radius values independant
    # for each point
    temp_dict = {}
    
    for (x,y), weight in list(grid_dict.items()):
        # len(radii_weight_list) should equal len(radius)
        ratio = weight/radius
        
        # used to determine how weighted a grid point is, given its distance from a direct point
        radii_weight_list = [weight/i for i in range(1,radius+2)]
        
        for xi in range(x-radius,x+radius+1):
            if xi > x_max:
                break
            elif xi < x_min:
                continue
            else:
                for yi in range(y-radius,y+radius+1):
                    if yi > y_max:
                        break
                    elif yi < y_min:
                        continue
                    else:
                        # calculate which ring of the radius the point belongs
                        ring = max(abs(xi-x),abs(yi-y))
                        
                        # dont add anything extra to the starting node (0th ring = (x,y))
                        if ring == 0:
                            continue
                            
                        if (xi,yi) in temp_dict:
                            temp_dict[xi,yi] += radii_weight_list[ring]
                        else:
                            temp_dict[xi,yi] = radii_weight_list[ring]
                            
    else:
        for idxs, weight in temp_dict.items():
            if idxs in grid_dict:
                grid_dict[idxs] += weight
            else:
                grid_dict[idxs] = weight

In [114]:
# calculate TL and BR coordinates for the grid
# trying out a simple min_lat,min_lon,max_lat,max_lon
def grid_idx_to_point(grid_idxs,square_pad):
    # separated logic to avoid isses with floats used as keys
    lat_idx,lon_idx = grid_idxs
    return lat_idx*square_pad-180,lon_idx*square_pad-180

def get_image_corners(grid_dict, square_pad):
    min_lat_idx  = np.inf
    min_lon_idx  = np.inf
    max_lat_idx  = -np.inf
    max_lon_idx  = -np.inf
    for grid_point in grid_dict:
        # lat
        if grid_point[0] < min_lat_idx :
            min_lat_idx  = grid_point[0]
        if grid_point[0] > max_lat_idx :
            max_lat_idx  = grid_point[0]
        # lon
        if grid_point[1] < min_lon_idx :
            min_lon_idx = grid_point[1]
        if grid_point[1] > max_lon_idx :
            max_lon_idx  = grid_point[1]
        
    # convert to coordinates
    # tl = max lat, min lon
    tl_coords = grid_idx_to_point((max_lat_idx,min_lon_idx),square_pad=square_pad)
    # tl = min lat, max lon
    br_coords = grid_idx_to_point((min_lat_idx,max_lon_idx),square_pad=square_pad)
    return (min_lat_idx,min_lon_idx), (max_lat_idx,max_lon_idx), tl_coords, br_coords



In [119]:
# from grid dict, convert to a grid matrix with just weights
def grid_dict_to_array(grid_dict, square_pad):
    tl_grid, br_grid, tl_coords, br_coords = get_image_corners(grid_dict, square_pad=square_pad)
    min_lat_idx, min_lon_idx = tl_grid
    max_lat_idx, max_lon_idx = br_grid
    
    grid_matrix = np.empty((max_lat_idx-min_lat_idx+1, max_lon_idx-min_lon_idx+1))
    for (lat_idx,lon_idx), weight in grid_dict.items():
        grid_matrix[lat_idx-min_lat_idx, lon_idx-min_lon_idx] = weight
        
    
    return (tl_coords, br_coords), grid_matrix





In [134]:
import cv2
import folium
import os

def calculate_image_matrix(df, 
                           weight, 
                           square_pad, 
                           morphology_mode=False,
                           radius=10, 
                           dirname="./image-results/",
                           filename="UNNAMED",
                           ext=".png", 
                           return_matrix=True):
    
    grid_dict = fit_all_points_to_grid(df,
                                       weight=weight,
                                       square_pad=square_pad
                                      )
    print("initial fit to grid")
    if not morphology_mode:
        add_radius_to_grid_dict(grid_dict,radius=radius,square_pad=square_pad)
    print("added radius")
    corners, grid_matrix = grid_dict_to_array(grid_dict, square_pad=square_pad)
    print("converted to matrix")
    
    # flip the matrix, because it was upside down... (latitude is inverted)
    grid_matrix = cv2.flip(grid_matrix, 0)
    
    print("flipped")
#     if morphology_mode:
#         # untested method to generate the point radii using gaussian blur
#         #   or maybe I will use morphological dilation for greyscale
#         #kernel = np.ones((5,5), dtype=np.uint8)
#         kernel = np.array([
#             [1,1,2,2,2,1,1],
#             [1,2,2,3,2,2,1],
#             [2,2,3,4,3,2,2],
#             [2,3,4,5,4,3,2],
#             [2,2,3,4,3,2,2],
#             [1,2,2,3,2,2,1],
#             [1,1,2,2,2,1,1]
#         ], dtype=np.uint8)
#         #kernel = np.ones((5,5),np.float32)/25
#         grid_matrix = cv2.filter2D(grid_matrix,-1,kernel)
#         #grid_matrix = cv2.dilate(grid_matrix, kernel2, iterations=1)
#         print("kernel stuff done")
        
    tlat,tlon,blat,blon = [round(c,7) for tup in corners for c in tup]
    # pad the bottom-right coordinates along one grid square
    blat -= square_pad
    blon += square_pad
    
    # something is up the ratio i believe
    print(tlat,tlon,blat,blon, sep="\n")
    new_dirname = dirname+filename+"/"
    try:
        os.mkdir(new_dirname)
    except FileExistsError:
        pass

    #ax.imshow(airport_matrix, cmap="inferno")
    cv2.imwrite(new_dirname+filename+ext, grid_matrix)
    print("Saved image")
    with open(new_dirname+filename+"_coords.txt","w") as outfile:
        outfile.write(f"TL,{tlat},{tlon}\n")
        outfile.write(f"BR,{blat},{blon}\n")
    
#     m = folium.Map([-37, 140], zoom_start=4)
#     folium.raster_layers.ImageOverlay(
#         image=grid_matrix,
#         bounds=[[tlat, tlon], [blat, blon]],
#         origin='upper', # due to flipping previously, it is not 'lower', but 'upper'
#         colormap=lambda x: (1, 0, 0, x) # reds
#         #mercator_project=True,
#     ).add_to(m)

#     m.save(dirname+filename+"_map.html")
#     print("Saved map")
    if return_matrix:
        coords = (tlat,tlon),(blat,blon)
        return coords, grid_matrix

    
    

In [135]:
SQUARE_PAD = 0.005

In [136]:
westfields = pd.read_csv("./cleaned-data/westfield-locations.csv")
westfields_coords, westfields_matrix = calculate_image_matrix(westfields, 
                                           weight=25, 
                                           radius=5, 
                                           square_pad=SQUARE_PAD, 
                                           filename="westfields",
                                           return_matrix=True
                                          )
#plt.imshow(westfields_matrix)

initial fit to grid
added radius
converted to matrix
flipped
-27.48
138.515
-38.050000000000004
153.34
Saved image


In [137]:

stations = pd.read_csv("./cleaned-data/stations.csv")
stations_coords, stations_matrix = calculate_image_matrix(stations,
                                         weight=25, 
                                         radius=5, 
                                         square_pad=SQUARE_PAD, 
                                         filename="stations",
                                         return_matrix=True
                                        )

initial fit to grid
added radius
converted to matrix
flipped
-32.025
149.69
-34.885000000000005
151.785
Saved image


In [138]:

airports = pd.read_csv("./cleaned-data/airports.csv")
airports_coords, airports_matrix = calculate_image_matrix(airports,
                                         weight=50, 
                                         radius=30, 
                                         square_pad=SQUARE_PAD, 
                                         filename="airports",
                                         return_matrix=True
                                        )

initial fit to grid
added radius
converted to matrix
flipped
-10.44
113.425
-42.995000000000005
153.715
Saved image


In [139]:
m=None

m = folium.Map([-35, 150], tiles="CartoDB dark_matter", zoom_start=7)
folium.raster_layers.ImageOverlay(
    image=stations_matrix,
    bounds= stations_coords,
    origin='upper', # due to flipping previously, it is not 'lower', but 'upper'
    colormap=lambda x: (1, 0, 0, x), # reds
    name="NSW Train Stations",
    mercator_project=False
).add_to(m)
folium.raster_layers.ImageOverlay(
    image=westfields_matrix,
    bounds=westfields_coords,
    origin='upper', # due to flipping previously, it is not 'lower', but 'upper'
    colormap=lambda x: (0, 1, 0, x), # greens
    name="Westfields",
    mercator_project=False
).add_to(m)
folium.Rectangle(stations_coords).add_to(m)
folium.Rectangle(westfields_coords,color="#FFFFFF").add_to(m)

for *position, name  in westfields[["lat","lon","name"]].values:
    folium.Marker(position, tooltip=name).add_to(m)
folium.LayerControl().add_to(m)
# airports doesnt work well with Folium
# folium.raster_layers.ImageOverlay(
#     image= airports_matrix,
#     bounds= airports_coords,
#     origin='upper', # due to flipping previously, it is not 'lower', but 'upper'
#     colormap=lambda x: (0, 0, 1, x), # blues
#     name="Airports"
# ).add_to(m)
m.save("./image-results/combined_map.html")